In [22]:
import torch
import torch.nn as nn

In [23]:
verbose = False
verboseprint = print if verbose else lambda *a, **k: None

In [24]:
"""
resnet for 1-d signal data, pytorch version

Shenda Hong, Oct 2019
"""

import numpy as np
from collections import Counter
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return (torch.tensor(self.data[index], dtype=torch.float), torch.tensor(self.label[index], dtype=torch.long))

    def __len__(self):
        return len(self.data)

class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            groups=self.groups)

    def forward(self, x):

        net = x

        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)

        net = self.conv(net)

        return net

class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):

        net = x

        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)

        net = self.max_pool(net)

        return net

class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super(BasicBlock, self).__init__()

        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.use_do = use_do

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.5)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.5)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=1,
            groups=self.groups)

        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):

        identity = x

        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            if self.use_do:
                out = self.do1(out)
        out = self.conv1(out)

        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        if self.use_do:
            out = self.do2(out)
        out = self.conv2(out)

        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)

        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)

        # shortcut
        out += identity

        return out

class ResNet1D(nn.Module):
    """

    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)

    Output:
        out: (n_samples)

    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes

    """

    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block, n_classes, downsample_gap=2, increasefilter_gap=4, use_bn=True, use_do=True, verbose=False):
        super(ResNet1D, self).__init__()

        self.verbose = verbose
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.use_do = use_do

        self.downsample_gap = downsample_gap # 2 for base model
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(in_channels=in_channels, out_channels=base_filters, kernel_size=self.kernel_size, stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = nn.ReLU()
        out_channels = base_filters

        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels

            tmp_block = BasicBlock(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=self.kernel_size,
                stride = self.stride,
                groups = self.groups,
                downsample=downsample,
                use_bn = self.use_bn,
                use_do = self.use_do,
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = nn.ReLU(inplace=True)
        # self.do = nn.Dropout(p=0.5)
        self.dense = nn.Linear(out_channels, n_classes)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        out = x

        # first conv
        if self.verbose:
            print('input shape', out.shape)
        out = self.first_block_conv(out)
        if self.verbose:
            print('after first conv', out.shape)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)

        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            if self.verbose:
                print('i_block: {0}, in_channels: {1}, out_channels: {2}, downsample: {3}'.format(i_block, net.in_channels, net.out_channels, net.downsample))
            out = net(out)
            if self.verbose:
                print(out.shape)

        # final prediction
        if self.use_bn:
            out = self.final_bn(out)
        out = self.final_relu(out)
        out = out.mean(-1)
        if self.verbose:
            print('final pooling', out.shape)
        # out = self.do(out)
        out = self.dense(out)
        if self.verbose:
            print('dense', out.shape)
        # out = self.softmax(out)
        if self.verbose:
            print('softmax', out.shape)

        return out

In [10]:
def solute_3_classes(solute, df):
    row = df.loc[df['SoluteName'] == solute].iloc[0]
    out = torch.tensor((row.Level1, row.Level2, row.Level3))
    return out


# solute_3_classes('hydrogen', df3)
def solute_TESA(solute, df):
    row = df[df['SoluteName'] == solute][:1]
    out = row[row.columns[17:26]]
    out = torch.tensor(out.values, dtype=torch.float)
    # print(f'type TESA: {out.dtype}')
    return out


# solute_TESA('hydrogen', df3)

def solvent_macro_props1(solvent, path_to_table):
    table = pd.read_table(path_to_table)
    row = table[table['Name'] == solvent]
    out = row[row.columns[2:]]
    out = torch.tensor(out.values, dtype=torch.float)
    return out

In [42]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Deleting charges species
filename = r'/Users/balepka/Yandex.Disk.localized/Study/Lab/Neural Network/MNSol-v2009_energies_v2.tsv'
with open(filename) as f:
    t = 0
    data = pd.read_table(f)
    df1 = pd.DataFrame(data)

df2 = df1.loc[df1['Charge'].isin([0])]

# Deleting solvent mixtures
filename2 = r'/Users/balepka/Yandex.Disk.localized/Study/Lab/Neural Network/MNSolDatabase_v2012/Solvent_properties.tsv'
with open(filename2) as f:
    data = pd.read_table(f, header=1)
    solvent_props = pd.DataFrame(data)
names = []
for name, count in df2['Solvent'].value_counts().items():
    row = solvent_props.loc[solvent_props['Name'] == name]
    values = np.array(row[['nD', 'alpha', 'beta', 'gamma', 'epsilon', 'phi', 'psi']])
    # print(f'{name} -> {count} -> {values.shape[0]}')
    if values.shape[0] == 0:
        print(name)
        names.append(name)
df3 = df2.loc[~df2['Solvent'].isin(names)]
df3 = df3[df3.SoluteName != 'water dimer']

## Creating table solvent - solute
Solvents = dict(df3['Solvent'].value_counts().items())

Solutes = dict(df3['SoluteName'].value_counts().items())
from collections import OrderedDict

Solvents = OrderedDict(df3['Solvent'].value_counts().items())
Solutes = OrderedDict(df3['SoluteName'].value_counts().items())
table_SS = pd.DataFrame(index=Solutes, columns=Solvents)

# testa = df3.loc[df3['Solvent'] == 'water'].loc[df3['SoluteName'] == 'hydrogen']['DeltaGsolv']
# testa
# if testa.empty:
#     print(1)
# else:
#     print(0)

for solute in Solutes:
    for solvent in Solvents:
        SS_row = df3.loc[df3['Solvent'] == solvent].loc[df3['SoluteName'] == solute]['DeltaGsolv']
        if SS_row.empty:
            pass
        else:
            table_SS[solvent][solute] = SS_row.item()

table_SS.to_csv('/Users/balepka/Yandex.Disk.localized/Study/Lab/Neural Network/Dataset preparation/table_SS.csv')

# table_SS.head()
table_SS.columns.tolist()
table_SS.index.tolist()
for solvent in table_SS.columns.tolist():
    for solute in table_SS.index.tolist():
        G_solv = table_SS[solvent][solute]
        if not pd.isna(G_solv):
            verboseprint(f's:{solvent} - {solute}: {G_solv}')
        else:
            verboseprint(f'NaN:{solvent} - {solute}: {G_solv}')


def test_sp(solvent):
    return torch.tensor(len(solvent))


def test_up(solute):
    return torch.tensor(len(solute))


## Creating SS Dataset
from torch.utils.data import Dataset


class SS_Dataset(Dataset):
    def __init__(self, table, solvent_props, solute_props, args = ((), ()), transform=None):
        self.table = table
        self.solvent_props = solvent_props
        self.solute_props = solute_props
        self.data = []
        self.transform = transform
        self.table = self.table.set_index('Unnamed: 0')
        self.sp, self.up = args
        for solvent in self.table.columns.tolist():
            for solute in self.table.index.tolist():
                G_solv = self.table[solvent][solute]
                if not pd.isna(G_solv):
                    self.data.append((solvent, solute, G_solv))

    def __getitem__(self, i):
        solvent, solute, G_solv = self.data[i]
        X1 = self.solvent_props(solvent, self.sp)
        X2 = self.solute_props(solute, self.up)
        #check dim
        # print(f'X1 - {X1.shape}')
        # print(f'X2 - {X2.shape}')
        verboseprint(f'X1 {X1.dtype}]')
        verboseprint(f'X2 {X2.dtype}]')
        X = torch.cat((X1, X2), 1)
        y = torch.tensor(G_solv)
        return X.float(), y.float()

    def __len__(self):
        return len(self.data)


table_v1 = pd.read_csv(
    '/Users/balepka/Yandex.Disk.localized/Study/Lab/Neural Network/Dataset preparation/table_SS_v1/table_SS_v1.csv')
# table_v1.head()

# table_v1.shape

from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF

table_v1 = pd.read_csv(
    '/Users/balepka/Yandex.Disk.localized/Study/Lab/Neural Network/Dataset preparation/table_SS_v1/table_SS_v1.csv')
path_props = '/Users/balepka/Yandex.Disk.localized/Study/Lab/Neural Network/MNSolDatabase_v2012/Solvent_properties1.tsv'
TESA_df = df3
args = (path_props, TESA_df)
dataset = SS_Dataset(table_v1, solvent_macro_props1, solute_TESA, args)
len_data = dataset.__len__()
val_data = len_data // 10
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len_data - val_data, val_data])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

octanol-water
diethylether-water
chloroform-water
heptane-water
isopropyltoluene
cyclohexane-water
benzene-water
ethylacetate-water
dichloroethane-water
carbontet-water
hexane-water
butanol-water
dibutylether-water
chlorobenzene-water
dibromoethane-water
nitrobenzene-water


In [50]:

import torch.optim as optim

def validate(model, val_loader):
    correct = 0
    total = 0
    all_MSE = 0
    loss = nn.MSELoss()
    with torch.no_grad():
        for vector, G_true in val_loader:
            vector, G_true = vector.to('cpu'), G_true.to('cpu')
            model.to('cpu')
            outputs = model(vector)
            total += G_true.size(0)
            all_MSE += loss(outputs.flatten(), G_true)
            # print(f'pr: {predicted}')
            # print(f'lab: {labels}')

    return all_MSE / total

In [26]:
def train(model, loss_function, optimizer,  epochs = 10, tag = "cifar10"):
  for epoch in range(epochs):
    hist_loss = 0
    for vector, G_true in tqdm(train_loader): # get bacth
        vector, G_true = vector.to(device), G_true.to(device)
        model.to(device)
        outputs = model(vector) # call forward inside

        loss = loss_function(outputs, G_true) # calculate loss
        loss.backward() # calculate gradients
        optimizer.step() # performs a single optimization step (parameter update).
        optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.

        hist_loss += loss.item() # For stat only

    # writer.add_scalar("Train/Loss", hist_loss / ( len(trainloader) ), epoch)
    # train_loss_data[tag].append(hist_loss/(len(trainloader)))
    # writer.close()
    # # train_accuracy = get_accuracy(model, trainloader)
    # writer.add_scalar("Train/Acc", train_accuracy, epoch)
    # # train_acc_data[tag].append(train_accuracy)
    # writer.close()

    # test_loss = 0
    # for images, labels in tqdm(testloader): # get bacth
    #     outputs = model(images) # call forward inside

    #     loss = loss_function(outputs, labels) # calculate loss
    #     test_loss += loss.item() # For stat only
    # writer.add_scalar("Test/Loss", test_loss / ( len(testloader) ), epoch)
    # # test_loss_data[tag].append(test_loss/(len(testloader)))

    # writer.close()

    accuracy = validate(model, val_loader)
    # writer.add_scalar("Test/Acc", accuracy, epoch)
    # test_acc_data[tag].append(accuracy)
    # writer.close()

    # accuracy = validate(model, testloader)
    print(print(f'epoch {epoch} -> {accuracy}'))
  return accuracy

In [51]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
in_channels = next(iter(train_loader))[0].shape[1]
model = ResNet1D(in_channels, base_filters=2, kernel_size=3, stride=2, groups=1, n_block=3, n_classes=1, use_bn=True, use_do=True, verbose=False)
model.to(device)  # Create model instance
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)  # Weight update
# reinit_tensorboard(clear_log = True)
my_acc =  train(model, nn.MSELoss(), optimizer, epochs = 10, tag = 'ResNET 1D' )
my_acc

100%|██████████| 25/25 [00:01<00:00, 13.14it/s]


epoch 0 -> 0.2731708586215973
None


100%|██████████| 25/25 [00:01<00:00, 13.58it/s]


epoch 1 -> 0.15287929773330688
None


100%|██████████| 25/25 [00:01<00:00, 13.60it/s]


...

100%|██████████| 25/25 [00:01<00:00, 12.72it/s]


epoch 8 -> 0.1521931290626526
None


100%|██████████| 25/25 [00:01<00:00, 13.78it/s]


epoch 9 -> 0.15166333317756653
None


tensor(0.1517)